In [ ]:
# This section is for the MEDIA publication 05/09/2023 2nd Dataset from Prateek
%run Medical_Utility.ipynb
%run Medical_IO.ipynb
%run Cyclekernel.ipynb
%run viewer.ipynb
%run viewer_3D.ipynb
from scipy.ndimage import zoom

def resize_3d_volume(volume, max_dim=256):
    original_shape = np.array(volume.shape)
    scaling_factors = max_dim / np.max(original_shape)
    new_shape = np.round(original_shape * scaling_factors).astype(int)
    resized_volume = zoom(volume, new_shape / original_shape, order=1)
    return resized_volume

def mass_srk_volume_segmentation(pathVol, pathOutVol, pathOutSeg, ext):
        '''
        @pathVol: path to volumes
        @pathOutVol: path to output shrinked volumes
        @pathOutSeg: path to output shrinked segmentations
        @ext: extension of the target files example: "nii"
        '''
        idx_arr = []
        filesVol = []
        outVol = []
        outSeg = []
        os.chdir(pathVol)
        for file in glob.glob("*."+ext):
            idx_arr.append(file.split('_')[1])
            filesVol.append(os.path.join(pathVol, file))
            outVol.append(os.path.join(pathOutVol, file.split('_')[0]+'_'+file.split('_')[1]+'_vol2_srk.nii'))
            outSeg.append(os.path.join(pathOutSeg, file.split('_')[0]+'_'+file.split('_')[1]+'_seg_srk.nii'))
        for i in range(len(filesVol)):
            vol = FileIO_MEDICAL.load_nii(filesVol[i])
            # pad vol with voxel witdh = 2
            assert(np.amin(vol) == 0.0)
            pad_dim0 = np.zeros((2, vol.shape[1], vol.shape[2]))
            vol_pad = np.concatenate((vol, pad_dim0), axis=0)
            vol_pad = np.concatenate((pad_dim0, vol_pad), axis=0)
            pad_dim1 = np.zeros((vol_pad.shape[0], 2, vol_pad.shape[2]))
            vol_pad = np.concatenate((vol_pad, pad_dim1), axis=1)
            vol_pad = np.concatenate((pad_dim1, vol_pad), axis=1)
            pad_dim2 = np.zeros((vol_pad.shape[0], vol_pad.shape[1], 2))
            vol_pad = np.concatenate((vol_pad, pad_dim2), axis=2)
            vol_pad = np.concatenate((pad_dim2, vol_pad), axis=2)
            seg = vol_pad.copy()
            seg[seg > 0.0] = 1
            seg[seg == 0.0] = 0
            # post process seg
            seg = Utility_MEDICAL.erode_binary_mask(seg, 1)
            dil_outer = Utility_MEDICAL.dilate_binary_mask(seg, 3)
            dil_inner = Utility_MEDICAL.dilate_binary_mask(seg, 2)
            srk_shape = Utility_MEDICAL.shrink_box(dil_outer)
            seg_srk = dil_inner[srk_shape[0]:srk_shape[1]+1, srk_shape[2]:srk_shape[3]+1, srk_shape[4]:srk_shape[5]+1]
            vol_srk = vol_pad[srk_shape[0]:srk_shape[1]+1, srk_shape[2]:srk_shape[3]+1, srk_shape[4]:srk_shape[5]+1]
            if (np.amax(seg_srk.shape) > 256):
                seg_srk = resize_3d_volume(seg_srk)
                vol_srk = resize_3d_volume(vol_srk)
            # normalize vol_srk
            vol_srk = (vol_srk - np.mean(vol_srk)) / np.std(vol_srk)
            vol_srk = vol_srk + abs(np.amin(vol_srk))
            vol_srk[seg_srk==0] = np.amin(vol_srk)-1
            # write out segmentation and shrinked volumes
            FileIO_MEDICAL.save_nii(vol_srk, outVol[i])
            FileIO_MEDICAL.save_nii(seg_srk, outSeg[i])

In [ ]:
# # loop through all files in a folder that ends with .nii
folder_path = 'E:/Data2/MEDIA_2ndDatasetpart3/cropped'
out_vol_path = 'E:/Data2/MEDIA_2ndDatasetpart3/shrinked/cropped'
out_seg_path = 'E:/Data2/MEDIA_2ndDatasetpart3/shrinked/segment'
# # files = os.listdir('E:/Data2/MEDIA_2ndDataset2part/shrinked/volume_variations/w_256_th0.1_dim1_dil0_fv')
# # nii_files = [file for file in files if file.endswith('.nii')]

# mass_srk_volume_segmentation(folder_path, out_vol_path, out_seg_path, 'nii')
# Utility_MEDICAL.mass_write_dat("E:/Data2/MEDIA_2ndDatasetpart3/shrinked/cropped", "E:/Data2/MEDIA_2ndDatasetpart3/shrinked/sup", "nii", "sup")
# labels = FileIO_MEDICAL.read_excel_labels('E:/Data2/MEDIA_2ndDatasetpart3/labels.xlsx')
# #labels = [1]
# Utility_MEDICAL.generate_tda_volumes('E:/Data2/MEDIA_2ndDatasetpart3/shrinked/sup', 'E:/Data2/MEDIA_2ndDatasetpart3/shrinked/volume_variations/w_256_th0.1_dim1_dil0_fv', 0.1, 1, 0, labels, True, 0, 2, False, (64,64,64))
# Utility_MEDICAL.unify_volume_size('E:/Data2/MEDIA_2ndDatasetpart3/shrinked/ss', 'E:/Data2/MEDIA_2ndDatasetpart3/shrinked/unifieddat_forvis', labels)


srk_vol = FileIO_MEDICAL.load_nii("E:/Data2/MEDIA_2ndDatasetpart3/shrinked/volume_variations/w_256_th0.1_dim1_dil0_fv/AI_0007_1_TDA.nii")
# out_vol = FileIO_MEDICAL.load_nii("E:/Data2/MEDIA_2ndDataset/shrinked/unifieddat_forvis/AI_0008_1_unifiedSize.nii")
# sup_vol = FileIO_MEDICAL.read_dat("E:/Data2/MEDIA_2ndDataset2part/shrinked/sup/AI_0043_vol2_sup.dat")
# # # # #FileIO_MEDICAL.save_nii(seg, 'E:/Data2/MEDIA_2ndDataset/kk.nii')
# viewer_3D.tmp2_viewer(out_vol, srk_vol)

print(srk_vol.shape)
print(np.amin(srk_vol), np.amax(srk_vol))
#srk_vol[srk_vol > 0.0] = 1.0
#srk_vol = srk_vol[:,20:,:380]

#srk_vol[srk_vol <= 10] = 0
#srk_vol[srk_vol > 10] = 1
# # # dil = Utility_MEDICAL.erode_binary_mask(srk_vol, 1)
# dil = Utility_MEDICAL.dilate_binary_mask(srk_vol, 2)
# srk = FileIO_MEDICAL.load_nii("E:/Data2/MEDIA_2ndDatasetpart3/cropped/AI_46_0.nii")
# srk[dil == 0.0] = 0.0
# srk = srk[:,100:,:135]

for i in range(srk_vol.shape[0]):
    viewer.imshow_(srk_vol[i,:,:])

#FileIO_MEDICAL.save_nii(srk, "E:/Data2/MEDIA_2ndDatasetpart3/cropped/AI_0_1.nii")

# for nii_file in nii_files:
#     print(nii_file)
#     vol = FileIO_MEDICAL.load_nii(os.path.join('E:/Data2/MEDIA_2ndDataset/shrinked/volume_variations/w_256_th0.1_dim1_dil0_fv', nii_file))
#     print(vol.shape)
#     print(np.amin(vol), np.amax(vol), np.prod(vol.shape), np.sum(vol == np.amin(vol)), np.sum(vol == np.amax(vol)))
#     print(len(np.unique(vol)))
#     print(np.sum(vol == np.amin(vol)) / np.prod(vol.shape))
#     print(np.sum(vol == np.amax(vol)) / np.prod(vol.shape))

    
    
# comment: the volume is not shrinked and max dim exceeds 256
# comment: sample 4 is in doubt
# pipeline:
# 1. manually refine all the volumes, most contains noise in the void
# 2. pad the volume in x, y, z directions with 2-voxel width apron [0, X]
# 3. shrink the volume to the smallest possible [-1, X]
# 4. resize the volume to max dim = 256 while keeping aspect ration intact
# 5. generate sup, [-X, 1]
# 6. generate tda
# 7. fit the tda filled data to 256 x 256 x 256 and shift the data range to [1, X] (not inverse)
# 8. visualize

In [ ]:
# print(np.amax(srk_vol), np.amin(srk_vol))
# print(np.sum(srk_vol == np.amax(srk_vol)) / np.prod(srk_vol.shape))
# print(np.sum(srk_vol == np.amin(srk_vol)) / np.prod(srk_vol.shape))
# print(len(np.unique(srk_vol)))

# print(np.amax(srk_vol2), np.amin(srk_vol2))
# print(np.sum(srk_vol2 == np.amax(srk_vol2)) / np.prod(srk_vol2.shape))
# print(np.sum(srk_vol2 == np.amin(srk_vol2)) / np.prod(srk_vol2.shape))
# print(len(np.unique(srk_vol2)))

# print()

# print(np.amax(sup_vol), np.amin(sup_vol))
# print(np.sum(sup_vol == np.amax(sup_vol)) / np.prod(sup_vol.shape))
# print(np.sum(sup_vol == np.amin(sup_vol)) / np.prod(sup_vol.shape))
# print(len(np.unique(sup_vol)))

# print(np.amax(sup_vol2), np.amin(sup_vol2))
# print(np.sum(sup_vol2 == np.amax(sup_vol2)) / np.prod(sup_vol2.shape))
# print(np.sum(sup_vol2 == np.amin(sup_vol2)) / np.prod(sup_vol2.shape))
# print(len(np.unique(sup_vol2)))

In [ ]:
%run Medical_Utility.ipynb
%run Medical_IO.ipynb
%run Cyclekernel.ipynb

# pers = FileIO_MEDICAL.read_pers_txt('E:/Data2/BreastMass_refine/sup/ISPY_1001_vol2_sup.dat.pers.txt')
# bnd = FileIO_MEDICAL.read_bnd_red_unifieddim('E:/Data2/BreastMass_refine/sup/ISPY_1001_vol2_sup.dat.bnd')
# seg_srk = FileIO_MEDICAL.load_nii("E:/Data2/BreastMass_refine/segmentations_srk/ISPY_1007_seg_srk.nii")

# labels = FileIO_MEDICAL.read_excel_labels('E:/Data2/BreastMass_refine/outcome.xlsx')
# pers_list = FileIO_MEDICAL.read_pers_txt_mass('E:/Data2/BreastMass_refine/sup', 'E:/Data2/BreastMass_refine/sup', 3)
# bnd_list = FileIO_MEDICAL.read_homo_unifieddim('E:/Data2/BreastMass_refine/sup', 'E:/Data2/BreastMass_refine/sup', 3)
# kernel_dist = Compute_CycleKernel.compute_cyckernel_wThreshold(bnd_list,3,pers_list,80,True,1.0)
# kernel_dist, filt_labels = Compute_CycleKernel.compute_cyckernel_wBornDeathLimit(
#     bnd_list, 3, pers_list, labels, ">=", -300, "dummy", 0, True, 1.0)



# vol = FileIO_MEDICAL.load_nii("E:/Data2/BreastMass_refine/volumes_srk/ISPY_1001_vol2_srk.nii")
# print(np.amin(vol), np.amax(vol))
# viewer_3D.tmp_viewer(vol)

# Utility_MEDICAL.mass_write_dat("E:/Data2/BreastMass_refine/volumes_srk_", "E:/Data2/BreastMass_refine/sup_", "nii", "sup")
# Utility_MEDICAL.mass_srk_volume_segmentation("E:/Data2/BreastMass_refine/volumes", "E:/Data2/BreastMass_refine/segmentations", "E:/Data2/BreastMass_refine/tumors_mask",
#                                              "E:/Data2/BreastMass_refine/volumes_srk_", "E:/Data2/BreastMass_refine/segmentations_srk", "E:/Data2/BreastMass_refine/tumors_srk", "nii")
# Utility_MEDICAL.align_tumor_segmentations("E:/Data2/BreastMass_refine/volumes", "E:/Data2/BreastMass_refine/tumors_mask", "E:/Data2/BreastMass_refine/tumors", "nii")
# FileIO_MEDICAL.copy_to_folder('E:/Data2/BreastMass', 'E:/Data2/BreastMass_refine/tumors_mask')
# path_vol = 'E:/Data2/BreastMass/ISPY_1001/registered/DCEMRI_1.nii'
# path_seg = 'E:/Data2/BreastMass/ISPY_1001/registered/seg_1.nii'
# vol = FileIO_MEDICAL.load_nii(path_vol)
# seg = FileIO_MEDICAL.load_nii(path_seg)
# dil = Utility_MEDICAL.dilate_binary_mask(seg, 5)
# srk_shape = Utility_MEDICAL.shrink_box(dil)
# seg_srk = seg[srk_shape[0]:srk_shape[1]+1, srk_shape[2]:srk_shape[3]+1, srk_shape[4]:srk_shape[5]+1]
# vol_srk = vol[srk_shape[0]:srk_shape[1]+1, srk_shape[2]:srk_shape[3]+1, srk_shape[4]:srk_shape[5]+1]
# vol_srk[seg_srk==0] = np.amin(vol)-1

# vol_srk = vol_srk + np.abs(np.amin(vol_srk)) + 1
# vol_srk = vol_srk * 10
# print(np.amin(vol_srk), np.amax(vol_srk))

# #seg_srk[seg_srk.shape[0]-1,:] = 1
# #layers = Utility_MEDICAL.divide_layers_around_core(seg_srk, 3)
# #layers = Utility_MEDICAL.divide_layers_against_plane(seg_srk, [seg_srk.shape[0]-1,0,0], 3)
# #FileIO_MEDICAL.write_dat(vol_srk, 'E:/Data2/BreastMass/ISPY_1001/registered/DCEMRI_1.dat')

# viewer_3D.tmp3_viewer(layers[0],layers[1],layers[2])

# # print(np.sum(seg==1) / np.prod(seg.shape))
# # print(np.sum(seg_srk==1) / np.prod(seg_srk.shape))
# # print(np.sum(dil==1) / np.prod(dil.shape))

# #viewer_3D.tmp_viewer(seg)
# #viewer_3D.tmp_viewer(dil)
# #viewer_3D.tmp_viewer(seg_srk)
# #viewer_3D.tmp_viewer(vol_srk)
# #print(np.amin(vol_srk), np.amax(vol_srk))

# # path_dat = 'E:/Data2/BreastMass/ISPY_1001/registered/DCEMRI_1.dat'
# path_bnd = 'E:/Data2/BreastMass/ISPY_1001/registered/DCEMRI_1.dat.bnd'
# bnd = FileIO_MEDICAL.read_bnd_red_unifieddim(path_bnd)
# # vol_srk = FileIO_MEDICAL.read_dat(path_dat)
# bnd_drw = Utility_MEDICAL.draw_on_volume(bnd, vol_srk.shape, 1, np.arange(5000,10000))
# viewer_3D.tmp2_viewer(vol_srk, bnd_drw)

# # path_cmp = 'E:/Data2/BreastMass_processed/1/whole/ISPY_1001_1.dat'
# # cmp = FileIO_MEDICAL.read_dat(path_cmp)
# # viewer_3D.tmp_viewer(cmp, opacity=1.0)

# ===== Naive Segmentation =====
# path_vol = 'E:/Data2/BreastMass_refine/volumes/ISPY_1122_vol2.nii'
# vol = FileIO_MEDICAL.load_nii(path_vol)
# # slic = vol[100,:]
# # seg = np.zeros(slic.shape, dtype=np.int32)
# # seg[slic > (np.amin(seg) + 10)] = 1
# # viewer.imshow_(seg)

# shape = vol.shape
# seg_auto = np.zeros(shape, dtype=np.int32)
# seg_auto[vol > (np.amin(vol) + 20)] = 1
# seg_auto = Utility_MEDICAL.erode_binary_mask(seg_auto, 1)
# seg_auto = Utility_MEDICAL.dilate_binary_mask(seg_auto, 7)

# #seg_auto[seg_auto.shape[0]-80,:] = 2
# seg_auto[seg_auto.shape[0]-80:seg_auto.shape[0],:,:] = 0

# #viewer_3D.tmp_viewer(seg_auto)
# FileIO_MEDICAL.save_nii(seg_auto, 'E:/Data2/BreastMass_refine/ISPY_1122_2.nii.gz')

# path_vol = 'E:/Data2/BreastMass_refine/volumes/ISPY_1163_vol2.nii'
# vol = FileIO_MEDICAL.load_nii(path_vol)
# rvol = vol[:,:,::-1]
# #print(vol.shape)
# vol = np.concatenate([vol, rvol], 1)
# # viewer.imshow_(vol[:,:,80])
# # viewer.imshow_(rvol[:,:,80])
# #viewer.imshow_(vol[90,:,:])
# #print(vol.shape)
# FileIO_MEDICAL.save_nii(vol, 'E:/Data2/BreastMass_refine/ISPY_1163_2.nii.gz')
# #viewer_3D.tmp_viewer(vol)

# # path_vol = 'E:/Data2/BreastMass_refine/ISPY_1161_mask2.nii'
# # vol = FileIO_MEDICAL.load_nii(path_vol)
# # print(vol.shape)
# # #viewer_3D.tmp_viewer(vol)
# # vol = vol[:,:150,:]
# # print(vol.shape)
# # #viewer_3D.tmp_viewer(vol)
# # FileIO_MEDICAL.save_nii(vol, 'E:/Data2/BreastMass_refine/ISPY_1161_2.nii.gz')

In [ ]:
%run Cyclekernel.ipynb
%run Medical_IO.ipynb
%run Medical_Utility.ipynb
import nrrd

#kernel_dist = Compute_CycleKernel.compute_cyckernel_wThreshold(bnd_list,3,pers_list,100,True,1.0)
#kernel_dist = Compute_CycleKernel.compute_cyckernel(bnd_list,3,True,1.0)

# labels = FileIO_MEDICAL.read_excel_labels('E:/Data2/BreastMass_refine/outcome_no1166.xlsx')
# Utility_MEDICAL.generate_tda_volumes('E:/Data2/BreastMass_refine/sup_', 'E:/Data2/BreastMass_refine/volume_variations/whole_256_th2_dim1_dil8_fv', 2.0, 1, 8, labels, True, 0, 2, False, (64,64,64))
# Utility_MEDICAL.generate_tda_volumes('E:/Data2/BreastMass_refine/sup_', 'E:/Data2/BreastMass_refine/volume_variations/whole_256_th2_dim2_dil8_fv', 2.0, 2, 8, labels, True, 0, 2, False, (64,64,64))

# target_id = 1001
# seg = FileIO_MEDICAL.load_nii("E:/Data2/BreastMass_refine/segmentations_srk/ISPY_"+str(target_id)+"_seg_srk.nii")
# tum = FileIO_MEDICAL.load_nii("E:/Data2/BreastMass_refine/tumors_srk/ISPY_"+str(target_id)+"_tum_srk.nii")
# vol = FileIO_MEDICAL.read_dat("E:/Data2/BreastMass_refine/sup/ISPY_"+str(target_id)+"_vol2_sup.dat")
# #vol = -vol
# vol = vol + np.abs(np.amin(vol)) + 1
# pers = FileIO_MEDICAL.read_pers_txt("E:/Data2/BreastMass_refine/sup/ISPY_"+str(target_id)+"_vol2_sup.dat.pers.txt")
# bnd = FileIO_MEDICAL.read_bnd_red_unifieddim("E:/Data2/BreastMass_refine/sup/ISPY_"+str(target_id)+"_vol2_sup.dat.bnd")
# bnd_filt = Compute_CycleKernel.filter_bnd_or_red(bnd, pers, 200)
# # print(len(bnd_filt[0]),len(bnd_filt[1]),len(bnd_filt[2]))
# #bnd_drw = Utility_MEDICAL.draw_on_volume(bnd_filt, vol.shape, 1, np.arange(0,len(bnd_filt[1])))
# bnd_drw = Utility_MEDICAL.draw_on_volume_ori_intensity(bnd_filt, vol, seg, tum, 10, 0, 1, np.arange(0,len(bnd_filt[1])), [500,1500])
# bnd_drw2 = Utility_MEDICAL.draw_on_volume_ori_intensity(bnd_filt, vol, seg, tum, 10, 0, 2, np.arange(0,len(bnd_filt[2])), [800,1200])
# #nrrd.write('E:/Data2/1202_label1.nrrd', bnd_drw*2+vol)

# # # print(np.amax(vol), np.amin(vol))
# # # print(np.amax(bnd_drw), np.amin(bnd_drw))

# # # kernel = Util_gen.generate_gaussian_kernel(3, 2, 3)
# # # smoothed = signal.convolve(bnd_drw, kernel,mode='same')
# # # viewer_3D.tmp2_viewer(vol, bnd_drw*0.6)
# # viewer_3D.tmp_viewer(vol, opacity=0.5)
# viewer_3D.tmp_viewer(vol, opacity=0.5)
# # viewer_3D.tmp5_viewer(vol, bnd_drw, bnd_drw2, tum*1900)

# # # === best pair of big difference between two groups: 0: 1001 1101 1117 1082 <-> 1: 1235 1220 1216 1202

In [ ]:
viewer_3D.tmp2_viewer(vol, bnd_drw*0.6)

In [ ]:
%run Medical_Utility.ipynb
%run Medical_IO.ipynb

labels = FileIO_MEDICAL.read_excel_labels('E:/Data2/BreastMass_refine/outcome.xlsx')
feat = [None] * 3
feat[0] = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim0_th100.dat')
feat[1] = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim1_th100.dat')
feat[2] = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim2_th100.dat')
# feat[0] = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim0_th30.dat')
# feat[1] = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim1_th30.dat')
# feat[2] = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim2_th30.dat')
# feat[0] = FileIO_MEDICAL.read_csv_1c1r('E:/Data2/BreastMass_refine/radiomic_features/radiomic_features_flag_normalize_false.csv')

Utility_MEDICAL.SVM_classifier(feat, 1, labels, 0.7, 20, True, True)


In [ ]:
FileIO_MEDICAL.write_dat(kernel_dist[0], 'E:/Data2/BreastMass_refine/cycdist_dim0_th80.dat')
FileIO_MEDICAL.write_dat(kernel_dist[1], 'E:/Data2/BreastMass_refine/cycdist_dim1_th80.dat')
FileIO_MEDICAL.write_dat(kernel_dist[2], 'E:/Data2/BreastMass_refine/cycdist_dim2_th80.dat')
#print(kernel_dist[2].shape)

In [ ]:
%run sliced_wasserstein.ipynb
%run Medical_Utility.ipynb
%run Medical_IO.ipynb

pers_list = FileIO_MEDICAL.read_pers_txt_mass('E:/Data2/BreastMass_refine/sup', 'E:/Data2/BreastMass_refine/sup', 3)
swp = sliced_wasserstein_python()
sw_dist = swp.compute_sw(pers_list, 100, "approximate")

In [ ]:
%run Medical_Utility.ipynb
%run Medical_IO.ipynb

# kernel_dist = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/cycdist_dim0_th30.dat')
# sw_dist = FileIO_MEDICAL.read_dat('E:/Data2/BreastMass_refine/sw_dim2_th30.dat')
# sw_dist = Utility_MEDICAL.normalize(sw_dist, 1.0)

labels = FileIO_MEDICAL.read_excel_labels('E:/Data2/BreastMass_refine/outcome_no1166.xlsx')
pers_list = FileIO_MEDICAL.read_pers_txt_mass('E:/Data2/BreastMass_refine/sup_', 'E:/Data2/BreastMass_refine/sup_', 3)
Utility_MEDICAL.probe_persistence(pers_list, labels, 0, ">=", -100)